# ADIA LAB CrunchDAO Notebook Submission

## 1. Setting up workspace cells

In [ ]:
## STEP 0
!pip3 install crunch-cli --upgrade

In [1]:
## STEP 2: Run this cell to load the data
import crunch
crunch = crunch.load_notebook(__name__)

loaded inline runner with module: <module '__main__'>


In [2]:
## STEP 3
!crunch --notebook setup stale-avi --token 4tT3eWdXumNoCG7b7FKZJLN17fYnww6gSl6yeHy4PdsYPeqTnrKiq49hRb4sdmm2
%cd stale-avi

stale-avi: already exists (use --force to override)
c:\GitHub\CrunchDAO\stale-avi


Aborted!


In [3]:
"""
IMPORTS I MAY USE
"""
import warnings
warnings.filterwarnings("ignore") # This is not advised in general, but it is used in this notebook to clean the presentation of results

import os, gc
import pandas as pd
import numpy as np
#from hyperopt import hp, fmin, tpe, Trials
#from hyperopt.pyll.base import scope
#from sklearn.metrics import roc_auc_score, roc_curve
#from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.notebook import tqdm
from joblib import dump, load
import joblib
from pathlib import Path
#import datatable as dtable
import typing

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [120]:
def train(X_train: pd.DataFrame, y_train: pd.DataFrame,
          model_directory_path: str = "resources") -> None:
    """
    Do your model training here.
    At each retrain this function will have to save an updated version of
    the model under the model_directiory_path, as in the example below.
    Note: You can use other serialization methods than joblib.dump(), as
    long as it matches what reads the model in infer().
    
    Args:
        X_train, y_train: the data to train the model.
        model_directory_path: the path to save your updated model
    
    Returns:
        None
    """
    #flush the memory
    ## For model construction
    #     alpha = 5
    # Num_hidden_limit = X_train.shape[0] / ( alpha * (X_train.shape[1] + 1) )
    # #alpha = 2 --> 800
    # #alpha = 5 --> 320
    # #alpha = 10 --> 160

    tf.random.set_seed(6741)

    train_samples = X_train.shape[0]
    n_features = X_train.shape[1] - 2
    #can this fit in memory?
    batch_size = 74267
    #needs to be small enough that it can fit in memory. 2500 x 461 should be good
    batch_size = 2500
    #batch_size = 1000

    ## do not include batch size when constructing input layer.
    model = tf.keras.models.Sequential([
        #tf.keras.layers.Flatten(input_shape = X_train.shape),
        tf.keras.layers.Input(shape = (None, train_samples, n_features), batch_size = batch_size, name = "Input_Layer"),
        tf.keras.layers.Dense(800, activation='swish', activity_regularizer = tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(600, activation='tanh', activity_regularizer = tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(400, activation='swish', activity_regularizer = tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(200, activation='tanh', activity_regularizer = tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(10, activation ='tanh'),
        tf.keras.layers.Dense(1, activation = None)
    ])

    #monitor = val_loss
    #es = EarlyStopping(monitor = 'val_action_AUC', min_delta = 1e-4, patience = 10, mode = 'max', 
    #                       baseline = None, restore_best_weights = True, verbose = 0)


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience = 2, restore_best_weights = True)
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)
    #my_callbacks = [ reduce_lr, early_stopping ]
    my_callbacks = [ early_stopping ]
    #if using KL Divergence as loss, don't add it as a metric
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate = 0.001) ,
        #optimizer = tf.keras.optimizers.experimental.SGD( learning_rate = 0.001),
        #loss = tf.keras.losses.BinaryCrossentropy(from_logits = True) ,
        #loss = tf.keras.losses.KLDivergence(),
        loss = tf.keras.losses.MeanSquaredError(),
        )

    # #Can do this if wanted
    # model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3) ,
    #     #loss = tf.keras.losses.BinaryCrossentropy(from_logits = True) ,
    #     loss = 'mse',
    #     metrics = [tf.keras.metrics.KLDivergence()]
    #     )

    # training the model
    print("training...")
    history = model.fit(
        x = X_train.iloc[:,2:],
        y = y_train.iloc[:,2:],
        epochs = 10,
        steps_per_epoch = int( np.ceil(train_samples / batch_size)),
        validation_split = 0.3,
        shuffle = True,
        workers = 8,
        use_multiprocessing = True,
        verbose = 1,
        callbacks = my_callbacks
     )
    print("Training complete.")


    ## SAVE THE MODEL
    # make sure that the train function correctly save the trained model
    # in the model_directory_path
    model_pathname = Path(model_directory_path) / "model.joblib"
    print(f"Saving model in {model_pathname}")
    joblib.dump(model, model_pathname)

    #Save history
    # convert the history.history dict to a pandas DataFrame:     
    # TODO: Figure out how to catch a PermissionError so this doesn't fail on the cloud.
    hist_df = pd.DataFrame(history.history) 
    hist_csv_file = 'resources/epoch_history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

def infer(X_test: pd.DataFrame,
          model_directory_path: str = "resources") -> pd.DataFrame:
    """
    Do your inference here.
    This function will load the model saved at the previous iteration and use
    it to produce your inference on the current date.
    It is mandatory to send your inferences with the ids so the system
    can match it correctly.
    
    Args:
        model_directory_path: the path to the directory to the directory in wich we will be saving your updated model.
        X_test: the independant  variables of the current date passed to your model.

    Returns:
        A dataframe (date, id, value) with the inferences of your model for the current date.
    """

    # loading the model saved by the train function at previous iteration
    model = joblib.load(Path(model_directory_path) / "model.joblib")
    
    # creating the predicted label dataframe with correct dates and ids
    y_test_predicted = X_test[["date", "id"]].copy()
    y_test_predicted["value"] = model.predict(X_test.iloc[:, 2:])

    return y_test_predicted

### Testing Locally - Part 1: Preliminary

In [104]:
from scipy.stats import spearmanr
# Getting the data
X_train, y_train, X_test = crunch.load_data()

download data\X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
already exists: file length match
download data\y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
already exists: file length match
download data\X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet
already exists: file length match
download data\y_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_test_reduced.parquet
already exists: file length match


In [118]:
def temporal_train_test_split(X_train_loc, y_train_loc, test_size=0.2):
    unique_dates = X_train_loc.date.unique()
    split_date = unique_dates[int(len(unique_dates)*(1-test_size))]
    X_train_local = X_train_loc[X_train_loc['date'] <= split_date]
    X_test_local = X_train_loc[X_train_loc['date'] > split_date]
    
    y_train_local = y_train_loc[y_train_loc['date'] <= split_date]
    y_test_local = y_train_loc[y_train_loc['date'] > split_date]
    
    return X_train_local, X_test_local, y_train_local, y_test_local

print("Splitting (X_train, y_train) in X_train_local, X_test_local, y_train_local, y_test_local")
X_train_local, X_test_local, y_train_local, y_test_local = temporal_train_test_split(
    X_train,
    y_train,
    test_size=0.20
)

print("X_train_local shape ", X_train_local.shape)
print("X_test_local shape ", X_test_local.shape)
print("y_train_local shape ", y_train_local.shape)
print("y_test_local shape ", y_test_local.shape)

Splitting (X_train, y_train) in X_train_local, X_test_local, y_train_local, y_test_local
X_train_local shape  (542200, 463)
X_test_local shape  (200470, 463)
y_train_local shape  (542200, 3)
y_test_local shape  (200470, 3)


In [121]:
#flush memory
tf.keras.backend.clear_session()
# Training. It may require a few minutes.
train(X_train_local, y_train_local)

print("#############")
print("Inference")
y_test_local_pred = infer(X_test_local, model_directory_path="resources")
score = spearmanr(y_test_local["y"], y_test_local_pred["value"])[0] * 100
print(f"Spearman's correlation {score}")

training...
Epoch 1/10
217/217 [==============================] - 41s 183ms/step - loss: 1.3136 - val_loss: 0.4554
Epoch 2/10
217/217 [==============================] - 38s 177ms/step - loss: 0.3933 - val_loss: 0.3667
Epoch 3/10
217/217 [==============================] - 40s 183ms/step - loss: 0.3524 - val_loss: 0.3490
Epoch 4/10
217/217 [==============================] - 36s 167ms/step - loss: 0.3427 - val_loss: 0.3426
Epoch 5/10
217/217 [==============================] - 36s 166ms/step - loss: 0.3395 - val_loss: 0.3403
Epoch 6/10
217/217 [==============================] - 36s 166ms/step - loss: 0.3381 - val_loss: 0.3388
Epoch 7/10
217/217 [==============================] - 36s 165ms/step - loss: 0.3375 - val_loss: 0.3390
Epoch 8/10
217/217 [==============================] - 36s 165ms/step - loss: 0.3372 - val_loss: 0.3378
Epoch 9/10
217/217 [==============================] - 36s 166ms/step - loss: 0.3368 - val_loss: 0.3386
Epoch 10/10
217/217 [==============================] - 36s 16

## Testing Model Locally Part 2: Actual Test Date

In [48]:
print("Remove unused data to release memory")
del X_train, y_train, X_test, X_train_local, X_test_local, y_train_local, y_test_local

Remove unused data to release memory


In [49]:
crunch.test(force_first_train=True, train_frequency=2)

ignoring cell #19: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 55)
ignoring cell #23: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 55)
ignoring cell #45: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 27)


09:48:18 forbidden library: requests
09:48:18 
09:48:18 running local test
09:48:18 internet access isn't restricted, no check will be done
09:48:18 


download data\X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
already exists: file length match
download data\y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
already exists: file length match
download data\X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet
already exists: file length match
download data\y_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_test_reduced.parquet
already exists: file length match


09:48:21 ---
09:48:21 loop: moon=269 train=True (1/5)
09:48:21 call: train


training...
Epoch 1/20
296/296 [==============================] - 25s 81ms/step - loss: 0.9868 - val_loss: 0.3759
Epoch 2/20
296/296 [==============================] - 25s 84ms/step - loss: 0.3579 - val_loss: 0.3431
Epoch 3/20
296/296 [==============================] - 25s 84ms/step - loss: 0.3415 - val_loss: 0.3382
Epoch 4/20
296/296 [==============================] - 24s 80ms/step - loss: 0.3383 - val_loss: 0.3367
Epoch 5/20
296/296 [==============================] - 24s 80ms/step - loss: 0.3374 - val_loss: 0.3365
Epoch 6/20
296/296 [==============================] - 24s 80ms/step - loss: 0.3371 - val_loss: 0.3366
Epoch 7/20
296/296 [==============================] - 24s 80ms/step - loss: 0.3372 - val_loss: 0.3361
Epoch 8/20
296/296 [==============================] - 24s 80ms/step - loss: 0.3372 - val_loss: 0.3367
Epoch 9/20
296/296 [==============================] - 25s 83ms/step - loss: 0.3369 - val_loss: 0.3370
Epoch 10/20
296/296 [==============================] - 24s 80ms/step -

...

Training complete.
Saving model in resources\model.joblib
128/128 [==============================] - 0s 3ms/step


10:08:42 ---
10:08:42 loop: moon=273 train=False (5/5)
10:08:42 call: infer


130/130 [==============================] - 0s 3ms/step


10:08:43 prediction_path=data\prediction.csv
10:08:43 duration: time=00:20:24
10:08:43 memory: before="5.77 GB" after="8.70 GB" consumed="2.93 GB"
10:08:43 local test succesfully run!


,date,id,value
0,269,c6e83eda40042dab1af117e195d542f00a417627e3173a...,-0.036208
1,269,97ae3194605438cbd2c59a3827f7c615dafa40d6cc3f42...,0.015091
2,269,310382927ec56f64c6f2f834fd320c9f732e26df639e67...,-0.004857
3,269,6e963f295f4ec1dc921be47638dba304f486ab2efd313c...,-0.031028
4,269,46cb6aa83fbd64a64a8a87d782476438abb658ce89b89c...,-0.042661
...,...,...,...
4127,273,26e5d74e138cf23f5d65aab46fdc59f6421e97ccf1ab8e...,-0.014090
4128,273,9dd2e69f186ef4eb076c646a8b182e936af2667793d143...,-0.003048
4129,273,002647639e3b83fd884eed0eddf72a702f15c5d70fb75d...,-0.002502
4130,273,7d840103d2370a80cc9b8376bfaf04b2aa5ff46bcbab03...,0.019638
